In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import datetime as dtt
# import xarray as xr
%matplotlib inline

In [ ]:
#read in all files with acii extension
ascii_files = glob.glob('*ASC*')
ascii_files

In [ ]:
#create list of columns, the += is appending the other lists
columns = ['datetime', 'ensemble_number', 'number_of_ensembles', 
           'pitch', 'roll', 'corrected_heading', 'adcp_temp']
columns += ['v_bt_x', 'v_bt_y', 'v_bt_z', 'v_bt_err', 
            'depth_snd', 'gga_alt', 'gga_dalt', 'gga_hdop',
            'depth_beam1', 'depth_beam2', 'depth_beam3', 'depth_beam4']
columns += ['total_elapsed_dist', 'total_elapsed_time', 'total_dist_n', 'total_dist_e', 'total_dist_mg']
columns += ['lat', 'lon', 'invalid', 'fixed_value_not_used']
columns += ['Q_middle', 'Q_top', 'Q_bot', 
            'start_shore_dist_est', 'start_dist', 'end_shore_dist_est', 'end_dist',
            'start_depth', 'end_depth']
columns += ['nbins', 'unit', 'vel_ref', 'intensity_units', 'intensity_scale_fac', 'sound_abs_fac']
columns

In [ ]:
#create list of columns for df
df_cols = ['depth', 'vmag', 'vdir', 'vx', 'vy', 'vz', 'verr', 'bs1', 'bs2', 'bs3', 'bs4', 'pctg', 'Q']

In [ ]:
def parse_header(f):
    #go through each row of the header, strips each piece and splits it from the rest
    row1 = next(f).strip().split()
    #checks if '20' is in the first row - i guess this is a marker for header info
    row1[0] = '20' + row1[0] if '20' not in row1[0] else row1[0]
    #in rows from start to row 6, convert to datetime
    dt = pd.datetime(*tuple(map(int, row1[:6])), int(int(row1[6]) * 1e4))
    #convert to another type of datetime (pandas to numpy)
    dt64 = np.datetime64(dt)
    #converts ensemble number to integer rows 7-9, maps this conversion to row 7-9
    ensemble_number, ne = map(int, row1[7:9])
    # concatenate datetime, ensembe number, and converts rows 9-end to be a float list
    data = [dt] + [ensemble_number, ne] + list(map(float, row1[9:]))
    #loops through range 0-4, appends next row (strip and split probably because dilemeter might change throughout)
    for i in range(4):
        #appends next row, converts to float
        #strip and split probably because dilemeter might change throughout
        data += list(map(float, next(f).strip().split()))
    #reassigns row 6, strips and splits it
    row6 = next(f).strip().split()
    #number of bins is integer of row6
    nbins = int(row6[0])
    #appends row 6 items 1-4 to data
    data += row6[1:4]
    #appends and converts to float items in 4-end in row 6
    data += list(map(float, row6[4:]))
    #returns ensumber number, data lsit, nbins, and datetime
    return ensemble_number, data, nbins, dt

def ascii2pd(ascii_file, make_geometries=False):
    # f is ascii file
    f = open(ascii_file)
    while True:
        #while there is a file to open, goes through each line, strips each line and splits it from the rest
        line = next(f).strip().split()
        #if length of that line is zero, continues to next step
        if len(line) == 0:
            continue
        else:
            #if not, loop breaks
            break
    #assigns file into to be integer of contents in line
    file_info = map(int, line)
    #pulls out each of these variables from file_info - order matters
    depth_cell_len, blank_after_transmit, adcp_depth_from_cn, n_depth_cells, n_pings, dt, mode = file_info
    
    #create two empty dictionaries
    data = {}
    dfs = {}
    
    try:
        while True:
            #while all conditions are true, run parse_header function to get
            # ensemble number, data, nbins, and datetime in that order
            n, d, nbins, dt = parse_header(f)
            # data[n] is assigned to data (d is panel)
            data[n] = d
            #creates a pandas dataframe and assigns column names
            df = pd.DataFrame([map(float, next(f).strip().split()) for b in range(nbins)], columns=df_cols)
            #df.index = df.depth
            #dfs[dt] (dataframe at some timestamp) is assigned by df in the dfs dictionary
            #timestamp is the key to access each part of the dictionary
            dfs[dt] = df
    #if there is stopiteration error, pass avoids code breaking
    except StopIteration:
        pass
    #creates pandas dataframe from the data dictionary, orients the data by index (datetime)
    df = pd.DataFrame.from_dict(data, orient='index')
    #assigns df columns to the column list created above
    df.columns = columns
    #converts string of datetime indices to datetime
    df.index = pd.to_datetime(df.datetime)
    #creates a geometry by grabbing lat and lon
    if make_geometries:
        df['geometry'] = [Point(r.lon, r.lat) for i, r in df.iterrows()]
        df['geometry'] = projectdf(df, '+init=epsg:4269', '+init=epsg:26715')
        df['X'] = [p.x for p in df.geometry]
        df['Y'] = [p.y for p in df.geometry]
    else:
        df['X'] = df.lon
        df['Y'] = df.lat
    
    # make a data panel of the velocity data
    pn = pd.Panel(dfs)
    # pn=xr.Dataset(d).to_array()
    # pn = xr.DataArray(dfs)
    # pn = pn.to_series()
    #returns pandas dataframe, dataframe dictionary, and panels
    return df, dfs,pn

def stack(df, pn, vmin, vmax, freq, make_geometries=False):
    #drops nans from each panel along axis one (across columns) and resamples down the indices
    #gets the mean and pnr becomes a copy
    pnr = pn.dropna(axis=1,how='all').resample(freq, axis=0).mean().copy()
    #assigns inds by using loc along all diminsions for veloctity x greater than min and less than max
    inds = (pnr.loc[:, :, 'vx'].values < vmin) | (pnr.loc[:, :, 'vx'].values > vmax)
    #creates another copy
    pnrs = pnr.copy()
    #masks data at inds and replaces values inplace for velocity x and y
    pnrs.loc[:, :, 'vx'].mask(inds, inplace=True)
    pnrs.loc[:, :, 'vy'].mask(inds, inplace=True)
    #slice dataframe by X,Y, and datetime, then resamples at variable freq and gets mean
    dfr = df[['X', 'Y', 'datetime']].resample(freq).mean()
    #if geometries is set to true in the function parameters, creates geometry column in dfr
    if make_geometries:
        dfr['geometry'] = [Point(r.X, r.Y) for i, r in dfr.iterrows()]
    #gets mean of velocity x and y across diminsions (time and ensemble)
    dfr['vx'] = pnrs.loc[:, :, 'vx'].mean()
    dfr['vy'] = pnrs.loc[:, :, 'vy'].mean()
    #drops nans down index
    dfr.dropna(axis=0, inplace=True)
    #returns resampled dataframe
    return dfr

### Read in each file and output the results to csv files

In [ ]:
outpath = 'output'
if not os.path.isdir(outpath):
    os.makedirs(outpath)

for ascii_file in ascii_files:
    if os.path.getsize(ascii_file) == 0:
        continue
    print(ascii_file)
    df, dfs,pn = ascii2pd(ascii_file)
    
    # display(pn)
    dfr = stack(df, pn, vmin=-1000, vmax=1000, freq='10s')
    
#     # flatten the panel to a dataframe
    dfall = pn.swapaxes(0, 2).to_frame()
    


### Dataframe of header information

### panel of backscatter data

In [ ]:
pn.axes;

In [ ]:
#first position.. ":" is for all timestamps, 
#second position ":10" is for everything up to ensemble 10
#last position is variable you want - in this case velocity y
pn.loc[:, :10., 'vy'];

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
#plots all timestamps (x) for ensemble numbers 0-10 (y) ,velocity y (value)
#vmin and vmax set the limit to the colormap/colorbar
plt.imshow(pn.loc[:, :10., 'vy'], vmin=-10, vmax=10) #, interpolation='None') #commented out interp.. looks smoother
plt.gca().set_aspect(100)
plt.colorbar()

### process single ADCP panel

In [ ]:
vmin, vmax = -1000, 1000 # valid range of velocities
freq = '10s' # resampling frequency

#takes panel, drops nans across timestamps, resamples at 10s for each ensemble gets mean and makes copy
pnr = pn.dropna(axis=1, how='all').resample(freq, axis=0).mean().copy()
#create inds for finding velocity x <min, >max for all timestamps and ensembles
inds = (pnr.loc[:, :, 'vx'].values < vmin) | (pnr.loc[:, :, 'vx'].values > vmax)
#make copy
pnrs = pnr.copy()
#apply mask to all timestamps and ensembles for velocity x and y and do it inplace
pnrs.loc[:, :, 'vx'].mask(inds, inplace=True)
pnrs.loc[:, :, 'vy'].mask(inds, inplace=True)

In [ ]:
#create backscatter variable
#all timestamps, ensemble 0-10, takes mean for all backscatter variables on resampled panel
bs = pnr.loc[:, :10, ['bs1', 'bs2', 'bs3', 'bs4']].mean(axis=2)
#all timestamps, ensemble 0-10, takes mean for all backscatter variables on original panel
bs = pn.loc[:, :10, ['bs1', 'bs2', 'bs3', 'bs4']].mean(axis=2)
bs.head()

In [ ]:
pn.loc[:, :10, ['bs1', 'bs2', 'bs3', 'bs4']]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
#creates plot for x(timestampe), y(ensemble), value (mean of backscatter variable in original panel)
#if you comment out the second bs variable, you can plot the resampled panel
#vmin and vmax set the limit to the colormap/colorbar
plt.imshow(bs, vmin=70, vmax=100)
plt.gca().set_aspect(100)
plt.colorbar()

### Plot velocity component across panel

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
#plots x(all timestamps), y(ensemble number 0-10), value (velocity y from resampled panel)
#vmin and vmax set the limit to the colormap/colorbar
plt.imshow(pnr.loc[:, :10, 'vy'], vmin=-10, vmax=10)
plt.gca().set_aspect(5)
plt.colorbar()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
#plots x(all timestamps), y(enemble numer 0-10), value (velocity y from masked resampled panel)
#vmin and vmax set the limit to the colormap/colorbar
plt.imshow(pnrs.loc[:, :10, 'vy'], vmin=-10, vmax=10)
plt.gca().set_aspect(5)
plt.colorbar()

### Slice panel to get dataframe of single measurement

In [ ]:
pn.loc[:'2022-03-03 19', :, 'vy'];

### reduce backscatter panel to dataframe 

In [ ]:
dfall = pn.swapaxes(0, 2).to_frame()
dfall.index.levels[0].name = 'depth_bin'
dfall.index.levels[1].name = 'datetime'
dfall.head()

### write dataframe to csv and then read it back in

In [ ]:
#     # write the csvs
with open('output/{}_alldata.csv'.format(ascii_file), "w") as f:
    dfall.to_csv(f)  # closes the file because old pands fails to        

with open('output/{}_header_info.csv'.format(ascii_file), "w") as f:
    df.to_csv(f, index=False)
# dfr.to_csv('output/{}_{}sampleddata.csv'.format(ascii_file, freq))
dfall.to_csv('alldata.csv')

In [ ]:
dfall2 = pd.read_csv('alldata.csv')

In [ ]:
dfall2.head()

### write csv of header information

In [ ]:
df.to_csv('header_info.csv', index=False) # write it without the index, since it duplicates the datetime column

In [ ]:
df